# DS Data Engineer (Data Eng)
Understand the data pipelines and business problems enough to be able to prescribe analytical solutions. Apply a diverse set of tactics including statistics and quantitative reasoning to solve problems as well as research and produce relevant product insights. Able to build ETL data pipelines and infrastructure to support Product and Data Science. Ability to understand business requirements and come up with scalable engineering solutions for data storage and retrieval. Understand and articulate data engineering and infrastructure decisions. Perform analyses of data trends to inform stakeholder decisions using a variety of visualization tools. Strong database and data visualization skills will be very valuable for this role.

- Solid Understanding of DS Units 1 & 3
    - Pandas
    - Databases
        - SQL (Postgresql)
        - No-SQL (MongoDB)
    - Graphing Libraries
        - Matplotlib
        - Plotly
        - Seaborn
        - Altair
- Build a Database Interface: MongoDB
    - Data Seeding: Mock Data
    - Filters & Projections
    - JSON Backup & Restore
- Build a Data Visualization Component: Plotly
    - Parameterized Abstraction Encapsulating Plotly


# MongoDB Basics
MongoDB Interface Class: see the `database.py` file in the `data_model` package

In [28]:
import os
import json
from typing import Sequence

from MonsterLab import Monster
from pymongo import MongoClient
from pandas import DataFrame
from dotenv import load_dotenv
import plotly.graph_objects as go
import plotly.express as px

## Load the environment variables from `.env` file
You should create your own `.env` file with the URL provided when you setup your Mongo account. See the `.env-example` file.

In [29]:
load_dotenv()

True

## Connect to the database
- URL: The URL given to us by MongoDB
    - Example: `mongodb+srv://<USER>:<PASS>@<CLUSTER>.<PROJECT_UUID>.mongodb.net`
- Database: This can be any name we like
    - Example: `MonsterLab`
- Collection: This can be any name we like
    - Example: `Monsters`

If we ever refer to a Database and/or Collection that doesn't exist, Mongo will create them for us as long as we have a Project setup and ready to go.

### Connection Info

In [30]:
url = os.getenv("MONGO_URL")
database = "MonsterLab"
collection = "Monsters"

### Instantiate the MongoClient with the connection info

In [31]:
db = MongoClient(url)[database][collection]

### Cleanup: Reset the collection by deleting all entries
This is not required if it is the first time connecting to this collection.

Passing an empty dictionary means delete all. If we only wanted to delete a subgroup we can pass a filter object instead. The filter needs to have one or more key/value pairs. See the section on Filters.

Delete all Dragons:
```
db.delete_many({"type": "Dragon"})
```

In [32]:
db.delete_many({})

### Example Data Point: Monster

In [33]:
monster = Monster()
monster

Name: Djinni
Type: Elemental
Level: 13
Rarity: Rank 1
Damage: 13d4+3
Health: 50.63
Energy: 52.14
Sanity: 51.52
Time Stamp: 2022-04-07 09:04:06

### Convert the Monster class instance into a dictionary

In [34]:
monster_dict = vars(monster)
monster_dict

{'type': 'Elemental',
 'name': 'Djinni',
 'level': 13,
 'rarity': 'Rank 1',
 'damage': '13d4+3',
 'time_stamp': '2022-04-07 09:04:06',
 'health': 50.63,
 'energy': 52.14,
 'sanity': 51.52}

### Insert the Monster into the Monsters collection

In [35]:
db.insert_one(monster_dict)

### Find all data points in the Monsters collection
Mongo returns an iterator, so we need to cast it to a list, or use it in a loop to print the data.

In [36]:
# Cast to a list
list(db.find())

[{'_id': ObjectId('624f0b7689bb67eb75589e12'),
  'type': 'Elemental',
  'name': 'Djinni',
  'level': 13,
  'rarity': 'Rank 1',
  'damage': '13d4+3',
  'time_stamp': '2022-04-07 09:04:06',
  'health': 50.63,
  'energy': 52.14,
  'sanity': 51.52}]

In [37]:
# Used in a loop
for monster in db.find():
    print(str(monster))

{'_id': ObjectId('624f0b7689bb67eb75589e12'), 'type': 'Elemental', 'name': 'Djinni', 'level': 13, 'rarity': 'Rank 1', 'damage': '13d4+3', 'time_stamp': '2022-04-07 09:04:06', 'health': 50.63, 'energy': 52.14, 'sanity': 51.52}


### Create Many Monsters

In [38]:
many_monsters = (vars(Monster()) for _ in range(999))

### Insert Many Monsters

In [39]:
db.insert_many(many_monsters)

### Checksum to see we have the new monsters added
We started with 1 then added 999 monsters. We should have a total of 1000 Monsters.

In [40]:
db.count_documents({})

1000

### Convert the Monsters Collection into a DataFrame

In [41]:
df = DataFrame(db.find())

### Save the data for ML Eng to use for training the ML model

In [42]:
df.to_csv("training_data.csv")

## Filters
It's common to only want a subgroup of the data in a collection. We could use pandas, but to do this efficiently we should use a Mongo filter query.

This does not affect the database in any way. The data is still in Mongo, it's just not downloaded for this query.

In [43]:
DataFrame(db.find({"type": "Dragon"}))

,_id,type,name,level,rarity,damage,time_stamp,health,energy,sanity
0,624f0b7689bb67eb75589e18,Dragon,Brass Drake,10,Rank 2,10d6,2022-04-07 09:04:06,60.85,57.90,59.18
1,624f0b7689bb67eb75589e1a,Dragon,Bronze Drake,3,Rank 0,3d2+1,2022-04-07 09:04:06,5.05,6.49,5.74
2,624f0b7689bb67eb75589e1d,Dragon,Faerie Dragon,12,Rank 1,12d4+1,2022-04-07 09:04:06,49.07,48.53,49.52
3,624f0b7689bb67eb75589e20,Dragon,Green Drake,5,Rank 3,5d8+3,2022-04-07 09:04:06,42.90,37.43,40.39
4,624f0b7689bb67eb75589e23,Dragon,Faerie Dragon,9,Rank 3,9d8+1,2022-04-07 09:04:06,75.82,72.54,71.28
...,...,...,...,...,...,...,...,...,...,...
165,624f0b7689bb67eb7558a1d5,Dragon,Faerie Dragon,3,Rank 1,3d4+1,2022-04-07 09:04:06,11.61,11.48,11.29
166,624f0b7689bb67eb7558a1de,Dragon,Gold Drake,15,Rank 3,15d8+2,2022-04-07 09:04:06,119.58,119.61,119.47
167,624f0b7689bb67eb7558a1e8,Dragon,Onyx Drake,10,Rank 0,10d2+1,2022-04-07 09:04:06,19.97,19.45,20.94
168,624f0b7689bb67eb7558a1eb,Dragon,Pseudodragon,9,Rank 0,9d2,2022-04-07 09:04:06,17.57,18.92,17.49


## Projections
For machine learning we only want the data that we intend to use for ML training. To do this we could use Pandas, but we can also use Mongo. It is far more efficient to use Mongo rather than download all the info then filter it. Let's see how to get rid of the `_id` and `time_stamp` columns with Mongo.

This does not affect the database in any way. The data is still in Mongo, it's just not downloaded for this query.

In [44]:
DataFrame(db.find(projection={"_id": False, "time_stamp": False}))

,type,name,level,rarity,damage,health,energy,sanity
0,Elemental,Djinni,13,Rank 1,13d4+3,50.63,52.14,51.52
1,Devilkin,Prince of Fear,9,Rank 1,9d4+4,34.14,34.08,36.89
2,Demonic,Imp,3,Rank 2,3d6+1,18.41,16.88,17.10
3,Fey,Mud Archfey,6,Rank 1,6d4+1,22.46,24.27,23.90
4,Devilkin,Goblin Knight,12,Rank 0,12d2+1,24.50,23.29,24.14
...,...,...,...,...,...,...,...,...
995,Devilkin,Kobold Villager,15,Rank 3,15d8,123.42,117.50,121.39
996,Demonic,Imp,9,Rank 0,9d2+2,17.91,18.29,18.98
997,Fey,Dust Archfey,12,Rank 2,12d6,72.52,69.57,71.98
998,Elemental,Ice Mephit,17,Rank 1,17d4,67.17,67.61,69.23


## A Filtered Projection
...And sometimes we want to use both filter and projection together.

In [45]:
DataFrame(db.find(
    {"type": "Dragon"},
    projection={"_id": False, "time_stamp": False},
))

,type,name,level,rarity,damage,health,energy,sanity
0,Dragon,Brass Drake,10,Rank 2,10d6,60.85,57.90,59.18
1,Dragon,Bronze Drake,3,Rank 0,3d2+1,5.05,6.49,5.74
2,Dragon,Faerie Dragon,12,Rank 1,12d4+1,49.07,48.53,49.52
3,Dragon,Green Drake,5,Rank 3,5d8+3,42.90,37.43,40.39
4,Dragon,Faerie Dragon,9,Rank 3,9d8+1,75.82,72.54,71.28
...,...,...,...,...,...,...,...,...
165,Dragon,Faerie Dragon,3,Rank 1,3d4+1,11.61,11.48,11.29
166,Dragon,Gold Drake,15,Rank 3,15d8+2,119.58,119.61,119.47
167,Dragon,Onyx Drake,10,Rank 0,10d2+1,19.97,19.45,20.94
168,Dragon,Pseudodragon,9,Rank 0,9d2,17.57,18.92,17.49


### Backup Database Monsters to JSON File
The auto generated "_id" field is a custom object type and can not be serialized to JSON.

The following cell should create a backup of our data and save it to "monsters.json" locally.

In [46]:
with open("monsters.json", "w") as file:
    json.dump(tuple(db.find(projection={"_id": False})), file)
db.count_documents({})

1000

### Delete All Monsters to Test our Backup & Restore Procedure

In [47]:
db.delete_many({})
db.count_documents({})

0

### Restore Database Monsters from JSON File

In [48]:
with open("mongo_backup.json", "r") as file:
    db.insert_many(json.load(file))
db.count_documents({})

1000

# Monster Graphs: Plotly

In [49]:
df_type = DataFrame(db.find(projection={"_id": False, "type": True}))
type_value_counts = df_type["type"].value_counts()
type_value_counts

Demonic      177
Dragon       175
Elemental    164
Undead       163
Devilkin     162
Fey          159
Name: type, dtype: int64

In [50]:
data = go.Pie(
    labels=type_value_counts.index,
    values=type_value_counts.values,
    hole=0.5,
    textinfo="percent+label",
    textfont={"size": 12},
    hoverinfo="text+label+value",
    textposition='inside',
    showlegend=False,
)
layout = go.Layout(
    title={
        "text": "Monster Counts by Types",
        "font": {"color": "white", "size": 24},
    },
    colorway=px.colors.qualitative.Antique,
    width=640,
    height=640,
    paper_bgcolor="#333333",
)
figure = go.Figure(data, layout)
figure.show()

### Wrap it in a Function
This will encapsulate our graphing code. That means we can change the implementation later without too much trouble.
For example, we could change to using Altair instead of Plotly, and the rest of the system wouldn't break.

In [51]:
def pie_chart(title: str, labels: Sequence, values: Sequence) -> go.Figure:
    return go.Figure(
        go.Pie(
            labels=labels,
            values=values,
            hole=0.5,
            textinfo="percent+label",
            textfont={"size": 12},
            hoverinfo="text+label+value",
            textposition='inside',
            showlegend=False,
        ),
        go.Layout(
            title={
                "text": title,
                "font": {"color": "white", "size": 24},
            },
            colorway=px.colors.qualitative.Antique,
            width=640,
            height=640,
            paper_bgcolor="#333333",
        ),
    )

In [52]:
# This should produce the same graph as above
pie_chart(
    "Monster Count by Type",
    type_value_counts.index,
    type_value_counts.values,
).show()

### New Data, New Graph, Same Function

In [53]:
df_dragon_rarity = DataFrame(
    db.find({"type": "Dragon"}, projection={"_id": False, "rarity": True}),
)
dragon_rarity_counts = df_dragon_rarity["rarity"].value_counts().sort_index()
dragon_rarity_counts

Rank 0    65
Rank 1    43
Rank 2    29
Rank 3    19
Rank 4    16
Rank 5     3
Name: rarity, dtype: int64

In [54]:
pie_chart(
    title="Dragon Count by Rarity",
    labels=dragon_rarity_counts.index,
    values=dragon_rarity_counts.values,
).show()